This notebook requires:
* the North Sea test case from the [getm-setups repository](https://github.com/getm-model/setups)
* the [TPXO9 atlas](https://www.tpxo.net/global/tpxo9-atlas) to calculate tides at the open boundaries

In [ ]:
import datetime
import os.path

import pygetm
import pygetm.legacy
import pygetm.input.tpxo

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

# Possible search directories
getm_setups_dirs = ('../../../getm-setups',)
igotm_data_dirs = ('/server/data', '../../../igotm/data')

# Determine effective directory by picking the first seach directory that exists
getm_setups_dir = next(filter(os.path.isdir, getm_setups_dirs))
igotm_data_dir = next(filter(os.path.isdir, igotm_data_dirs))

# Specific paths to forcing
tpxo_dir = os.path.join(igotm_data_dir, 'TPXO9')
era_path = os.path.join(igotm_data_dir, 'ERA-interim/2016.nc')

In [ ]:
# Load bathymetry and create simulation
domain = pygetm.legacy.domain_from_topo(os.path.join(getm_setups_dir, 'NorthSea/Topo/NS6nm.v01.nc'), nlev=30, z0_const=0.001)
pygetm.legacy.load_bdyinfo(domain, os.path.join(getm_setups_dir, 'NorthSea/bdyinfo.dat'), type_2d=-4)
sim = pygetm.Simulation(domain, runtype=pygetm.BAROTROPIC_2D, apply_bottom_friction=True, advection_scheme=pygetm.AdvectionScheme.HSIMT, log_level='ERROR')

# Plot bathymetry
fig, ax = matplotlib.pyplot.subplots()
ax.pcolormesh(domain.T.loni, domain.T.lati, domain.T.H.ma);

In [ ]:
fig, ax = matplotlib.pyplot.subplots(figsize=(12,12))
pc = ax.pcolormesh(domain.lon, domain.lat, domain.mask, shading='auto');
fig.colorbar(pc)
ax.plot(domain.open_boundaries.lon, domain.open_boundaries.lat, 'ow', mec='k', ms=2);
ax.set_title('mask on supergrid');

In [ ]:
# Tidal boundary forcing from TPXO
domain.open_boundaries.z.set(pygetm.input.tpxo.get(domain.open_boundaries.lon, domain.open_boundaries.lat, root=tpxo_dir))
domain.open_boundaries.u.set(pygetm.input.tpxo.get(domain.open_boundaries.lon, domain.open_boundaries.lat, variable='u', root=tpxo_dir))
domain.open_boundaries.v.set(pygetm.input.tpxo.get(domain.open_boundaries.lon, domain.open_boundaries.lat, variable='v', root=tpxo_dir))

# Meteorology from ERA
era_kwargs = {'preprocess': lambda ds: ds.isel(time=slice(4, -4))}
sim.airsea.tcc.set(pygetm.input.from_nc(era_path, 'tcc', **era_kwargs))
sim.airsea.t2m.set(pygetm.input.from_nc(era_path, 't2m', **era_kwargs) - 273.15)
sim.airsea.d2m.set(pygetm.input.from_nc(era_path, 'd2m', **era_kwargs) - 273.15)
sim.airsea.sp.set(pygetm.input.from_nc(era_path, 'sp', **era_kwargs))
sim.airsea.u10.set(pygetm.input.from_nc(era_path, 'u10', **era_kwargs))
sim.airsea.v10.set(pygetm.input.from_nc(era_path, 'v10', **era_kwargs))
sim.sst = sim.airsea.t2m

In [ ]:
# Set up initial plot of elevation (to be animated later)
start_date = datetime.datetime(2016, 1, 1)
sim.start(start_date, timestep=60, split_factor=30)
fig, ax = matplotlib.pyplot.subplots()
pc = ax.pcolormesh(domain.T.loni, domain.T.lati, domain.T.z.ma, vmin=-2.5, vmax=2.5)
sc = ax.scatter(domain.open_boundaries.lon, domain.open_boundaries.lat, c=domain.open_boundaries.z.all_values, edgecolors='k', vmin=-2.5, vmax=2.5, s=2, linewidth=.2)
cb = fig.colorbar(sc)
cb.set_label('elevation (m)')
title = ax.set_title(start_date.strftime('%Y-%m-%d %H:%M'))

In [ ]:
stop_date = start_date + datetime.timedelta(days=30)

itime = 0
while sim.time < stop_date:
    sim.advance()
    itime += 1
    if itime % 15 == 0:
        # Update plot
        title.set_text(sim.time.strftime('%Y-%m-%d %H:%M'))
        sc.set_array(domain.open_boundaries.z.all_values)
        pc.set_array(domain.T.z.ma.ravel())

        fig.canvas.draw()

sim.finish()